# Session 3: RAG (Retrieval-Augmented Generation) — Interactive Demos

**Audience:** Banking Technologists  
**Duration:** ~90 minutes of live demos  
**Embedding Model:** `sentence-transformers/all-MiniLM-L6-v2` (free, local, no API key required)  
**LLM:** Claude (Anthropic API — only needed for generation demos)

---

## What You'll Build

A complete banking RAG system from scratch, one stage at a time:

```
Demo 1: Embeddings — What are they? Why do they work?
Demo 2: Chunking — How to split banking documents
Demo 3: Vector Search — Find relevant policy sections
Demo 4: Hybrid Search — Vector + keyword for regulatory terms
Demo 5: Complete RAG Pipeline — End-to-end with Claude
Demo 6: RAG Failure Modes — Debug poor retrieval
Demo 7: Advanced Retrieval — Re-ranking, HyDE, Contextual
Demo 8: Evaluation — Measure your RAG quality
```

---
## Setup — Run This First

In [ ]:
# Install required packages (run once, ~2 minutes)
!pip install -q sentence-transformers chromadb anthropic rank-bm25 rich numpy

In [ ]:
import os
import json
import time
import numpy as np
from typing import List, Dict
from rich.console import Console
from rich.table import Table
from rich.panel import Panel

console = Console()

# ── Load Embedding Model (local, free, ~80MB download once) ──────────────
from sentence_transformers import SentenceTransformer

print("Loading embedding model (first run downloads ~80MB)...")
EMBED_MODEL = SentenceTransformer('all-MiniLM-L6-v2')
print(f"✅ Model loaded: all-MiniLM-L6-v2")
print(f"   Dimensions: {EMBED_MODEL.get_sentence_embedding_dimension()}")
print(f"   Max tokens: 256 per chunk")
print(f"   Cost: FREE (runs locally)")

def embed(text: str) -> np.ndarray:
    """Embed a single text. Returns numpy array."""
    return EMBED_MODEL.encode(text, normalize_embeddings=True)

def embed_batch(texts: List[str]) -> np.ndarray:
    """Embed multiple texts efficiently."""
    return EMBED_MODEL.encode(texts, normalize_embeddings=True, show_progress_bar=True)

def cosine_sim(a: np.ndarray, b: np.ndarray) -> float:
    """Cosine similarity. Returns float 0-1."""
    return float(np.dot(a, b))  # Already normalized → dot = cosine

print("\n✅ Setup complete! Ready to build banking RAG.")

In [ ]:
# ── Banking Policy Data (used throughout all demos) ──────────────────────
# Simulates realistic KYC/AML policy manual chunks

BANKING_POLICY_CHUNKS = [
    # KYC / Account Opening
    "Section 3.4: Business Account Opening Requirements. All business accounts require: "
    "(1) Articles of incorporation or business registration certificate, "
    "(2) Employer Identification Number (EIN) from IRS, "
    "(3) FinCEN Beneficial Ownership Certification for all owners with 25% or more stake, "
    "(4) Government-issued photo ID for all owners above 25% threshold, "
    "(5) Business license where required by state law.",

    "Section 3.5: Individual Account Opening Requirements. Retail accounts require: "
    "(1) Government-issued photo ID (passport, driver's license, or state ID), "
    "(2) Proof of address dated within 90 days (utility bill, bank statement, lease), "
    "(3) Social Security Number or ITIN for US persons, "
    "(4) Date of birth for OFAC screening. "
    "Non-resident aliens require Form W-8BEN.",

    # AML / CTR / SAR
    "Section 4.2: Currency Transaction Report (CTR) Filing Requirements. "
    "Financial institutions must file a CTR (FinCEN Form 104) for cash transactions "
    "exceeding $10,000 in a single business day. Multiple transactions that aggregate "
    "to more than $10,000 (structuring detection) must also be reported. "
    "CTR must be filed within 15 calendar days of the transaction.",

    "Section 5.1: Politically Exposed Persons (PEP) Enhanced Due Diligence. "
    "PEPs are defined as individuals who hold or have held a prominent public position, "
    "including foreign government officials, senior executives of state-owned enterprises, "
    "and their immediate family members. All PEP accounts require: "
    "(1) Senior management approval before account opening, "
    "(2) Annual review regardless of transaction activity, "
    "(3) Enhanced transaction monitoring with lower alert thresholds.",

    "Section 6.3: Suspicious Activity Report (SAR) Filing Obligations. "
    "A SAR must be filed when the bank knows, suspects, or has reason to suspect "
    "a transaction involves funds from illegal activity, is designed to evade BSA "
    "reporting requirements (structuring), lacks a lawful purpose, or involves "
    "transactions of $5,000 or more. SARs must be filed within 30 days of detection, "
    "or 60 days if no suspect is identified. SAR filings are confidential.",

    # Wire Transfers
    "Section 4.5: International Wire Transfer Monitoring. "
    "All international wire transfers must include originator and beneficiary information "
    "per the Travel Rule (31 CFR 103.33). Wires to FATF high-risk jurisdictions require "
    "enhanced due diligence. Transfers over $50,000 to non-established correspondents "
    "require Compliance approval. Same-day OFAC screening is mandatory for all wires.",

    # Record Keeping
    "Section 8.2: KYC Record Retention Policy. "
    "All customer identification documents must be retained for a minimum of 5 years "
    "from account opening, or 5 years after account closure, whichever is longer. "
    "CTR and SAR records must be retained for 5 years from filing date. "
    "Wire transfer records must be retained for 5 years per the Travel Rule. "
    "Records must be retrievable within 72 hours of regulatory request.",

    # Risk Ratings
    "Section 9.1: Customer Risk Rating Framework. "
    "Customers are assigned a risk rating of Low, Medium, or High at onboarding "
    "and reviewed annually. High-risk factors include: PEP status, high-risk geography, "
    "cash-intensive business, complex ownership structure, prior SAR filing. "
    "High-risk customers require quarterly review. Risk ratings drive monitoring "
    "alert thresholds and CDD refresh frequency.",

    # Overdraft
    "Section 11.4: Overdraft Fee Waiver Policy. "
    "Overdraft fees of $35 per occurrence may be waived under the following conditions: "
    "(1) First overdraft in any rolling 12-month period, "
    "(2) Customer has maintained account for 12+ months in good standing, "
    "(3) Overdraft amount is less than $25. "
    "Waivers must be approved by branch manager or call center supervisor. "
    "Maximum of one courtesy waiver per 12-month period.",

    # Capital
    "Section 2.1: Capital Adequacy Overview (Basel III). "
    "The bank maintains Common Equity Tier 1 (CET1) capital ratio above the regulatory "
    "minimum of 4.5% plus the 2.5% Capital Conservation Buffer, for an effective minimum "
    "of 7.0%. Internal target is 10.5% CET1. Total capital ratio must exceed 10.5% "
    "(8% minimum + 2.5% buffer). Stress testing under DFAST is conducted annually.",

    # OFAC
    "Section 7.1: OFAC Sanctions Compliance. "
    "The Office of Foreign Assets Control (OFAC) administers and enforces economic "
    "sanctions programs. All customers and transactions must be screened against OFAC "
    "SDN (Specially Designated Nationals) list before processing. "
    "Any potential match must be escalated to Compliance within 1 hour. "
    "Transactions with sanctioned entities must be blocked and reported to OFAC "
    "within 10 business days via the OFAC reporting portal.",
]

CHUNK_METADATA = [
    {"section": "3.4", "topic": "account_opening", "department": "retail", "doc_id": "POL-KYC-2024"},
    {"section": "3.5", "topic": "account_opening", "department": "retail", "doc_id": "POL-KYC-2024"},
    {"section": "4.2", "topic": "ctr_filing",      "department": "compliance", "doc_id": "POL-AML-2024"},
    {"section": "5.1", "topic": "pep_edd",          "department": "compliance", "doc_id": "POL-AML-2024"},
    {"section": "6.3", "topic": "sar_filing",       "department": "compliance", "doc_id": "POL-AML-2024"},
    {"section": "4.5", "topic": "wire_transfers",   "department": "operations", "doc_id": "POL-AML-2024"},
    {"section": "8.2", "topic": "record_retention", "department": "compliance", "doc_id": "POL-KYC-2024"},
    {"section": "9.1", "topic": "risk_rating",      "department": "compliance", "doc_id": "POL-AML-2024"},
    {"section": "11.4","topic": "overdraft",        "department": "retail",     "doc_id": "POL-OPS-2024"},
    {"section": "2.1", "topic": "capital",          "department": "risk",       "doc_id": "POL-CAP-2024"},
    {"section": "7.1", "topic": "ofac_sanctions",   "department": "compliance", "doc_id": "POL-AML-2024"},
]

print(f"✅ Loaded {len(BANKING_POLICY_CHUNKS)} banking policy chunks")
print(f"   Topics: {list(set(m['topic'] for m in CHUNK_METADATA))}")

---
## Demo 1: What Are Embeddings?

**Concept:** Text → vector of numbers that captures meaning  
**Banking context:** Why `KYC` and `Know Your Customer` should return the same policy

In [ ]:
# ── 1A: Embed some text and inspect the vector ────────────────────────────
sample_text = "KYC wire transfer threshold"
vector = embed(sample_text)

print(f"Text:      '{sample_text}'")
print(f"Dimensions: {len(vector)}")
print(f"First 10 values: {vector[:10].round(4)}")
print(f"Range: [{vector.min():.3f}, {vector.max():.3f}]")
print(f"Norm (should be 1.0 after normalization): {np.linalg.norm(vector):.4f}")

In [ ]:
# ── 1B: Semantic similarity between banking terms ─────────────────────────
# The key insight: similar MEANING → similar VECTOR → high similarity score

term_pairs = [
    # (text_a, text_b, expected_relationship)
    ("KYC requirements",               "Know Your Customer procedures",    "Synonyms"),
    ("AML screening",                  "anti-money laundering check",      "Synonyms"),
    ("Currency Transaction Report",    "CTR filing",                       "Acronym"),
    ("suspicious activity report",     "SAR filing",                       "Acronym"),
    ("wire transfer limit",            "wire transfer threshold",          "Near-synonyms"),
    ("KYC requirements",               "mortgage interest rate",           "Unrelated"),
    ("AML compliance",                 "recipe for chocolate cake",        "Completely different"),
    ("Basel III capital ratio",        "CET1 requirement",                 "Related concepts"),
    ("politically exposed person",     "PEP enhanced due diligence",       "Related"),
]

console.print("[bold cyan]Semantic Similarity Between Banking Terms[/bold cyan]\n")

table = Table(show_header=True)
table.add_column("Text A",        style="cyan",   max_width=35)
table.add_column("Text B",        style="yellow", max_width=35)
table.add_column("Similarity",    style="green",  justify="right")
table.add_column("Relationship",  style="white")
table.add_column("Verdict",       style="bold")

for a, b, rel in term_pairs:
    sim = cosine_sim(embed(a), embed(b))
    if sim > 0.75:
        verdict = "[green]✓ CLOSE[/green]"
    elif sim > 0.50:
        verdict = "[yellow]~ RELATED[/yellow]"
    else:
        verdict = "[red]✗ DISTANT[/red]"
    table.add_row(a[:35], b[:35], f"{sim:.3f}", rel, verdict)

console.print(table)
print("\n📌 Key insight: Embeddings capture meaning, not just keywords.")
print("   'KYC' ≈ 'Know Your Customer' → same policy retrieved for both queries")

In [ ]:
# ── 1C: Visualize embedding space (2D projection) ─────────────────────────
# Cluster of banking terms visually proves semantic grouping

try:
    from sklearn.decomposition import PCA
    import matplotlib.pyplot as plt
    import matplotlib
    matplotlib.rcParams['figure.figsize'] = (12, 8)

    banking_terms = [
        # Compliance cluster
        "KYC", "AML", "BSA", "OFAC", "SAR", "CTR",
        # Capital cluster
        "CET1", "Basel III", "RWA", "DFAST", "capital ratio",
        # Retail cluster
        "mortgage", "overdraft", "credit card", "savings account",
        # Risk cluster
        "credit risk", "market risk", "counterparty", "stress test",
    ]
    colors = (["#e74c3c"] * 6 +   # Red: Compliance
              ["#3498db"] * 5 +   # Blue: Capital
              ["#2ecc71"] * 4 +   # Green: Retail
              ["#f39c12"] * 4)    # Orange: Risk
    labels = (["Compliance"] * 6 + ["Capital"] * 5 +
               ["Retail"] * 4    + ["Risk"] * 4)

    vectors = embed_batch(banking_terms)

    pca = PCA(n_components=2)
    coords = pca.fit_transform(vectors)

    fig, ax = plt.subplots()
    for i, (term, (x, y), color) in enumerate(zip(banking_terms, coords, colors)):
        ax.scatter(x, y, color=color, s=100, zorder=2)
        ax.annotate(term, (x, y), textcoords="offset points",
                    xytext=(5, 5), fontsize=9)

    from matplotlib.patches import Patch
    legend = [Patch(color=c, label=l) for c, l in
              zip(["#e74c3c","#3498db","#2ecc71","#f39c12"],
                  ["Compliance","Capital","Retail","Risk"])]
    ax.legend(handles=legend, loc="upper right")

    ax.set_title("Banking Terms in Embedding Space\n(2D PCA projection of 384-dim vectors)",
                 fontsize=13)
    ax.set_xlabel("Principal Component 1")
    ax.set_ylabel("Principal Component 2")
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    print("📌 Notice how compliance terms (red) cluster together,")
    print("   even though they were never explicitly grouped!")

except ImportError:
    print("Run: pip install scikit-learn matplotlib  to see the 2D visualization")

---
## Demo 2: Chunking Banking Documents

**Concept:** Why chunking strategy dramatically affects retrieval quality  
**Banking context:** KYC policy manual — what happens with wrong chunk sizes

In [ ]:
# ── 2A: Simulate a realistic policy document ──────────────────────────────
SAMPLE_POLICY_DOCUMENT = """
KYC/AML POLICY MANUAL — CHAPTER 3: ACCOUNT OPENING

3.1 Overview
All customer accounts must complete the Customer Identification Program (CIP) before
account activation. This program ensures compliance with the Bank Secrecy Act and
FinCEN Customer Due Diligence (CDD) rules effective May 2018.

3.2 Risk-Based Approach
The bank applies a risk-based approach to customer due diligence. Higher-risk customers
receive enhanced scrutiny. Risk factors include: geography, business type, transaction
volume, and PEP status. Risk ratings are Low, Medium, or High.

3.3 Individual Account Requirements
For individual retail accounts, the following documents are required:
Government-issued photo identification (passport, state ID, or driver's license).
Secondary ID required if primary is expired. Proof of address within 90 days.
Social Security Number required for US persons for tax reporting (FATCA).
Non-resident aliens must complete IRS Form W-8BEN.

3.4 Business Account Requirements
For business accounts, enhanced documentation is required under FinCEN CDD Rule:
Articles of incorporation or business registration certificate from state of formation.
Employer Identification Number (EIN) from IRS Form SS-4 or IRS confirmation letter.
FinCEN Beneficial Ownership Certification Form identifying all owners with 25% or
greater ownership stake and one control person (regardless of ownership percentage).
Government-issued photo ID for each beneficial owner above the 25% threshold.
Business license where required by applicable state or local law.
For foreign businesses: apostilled formation documents and legal opinion where required.

3.5 Trust Account Requirements
Trust accounts require the trust agreement or certificate of trust showing trustees.
All trustees must complete individual ID verification. Grantor trusts require grantor ID.
Corporate trustees must complete business account requirements listed in section 3.4.

3.6 Account Opening Approval
Standard accounts: opened by branch staff upon document completion.
High-risk accounts (PEPs, cash-intensive businesses): require BSA Officer approval.
Foreign accounts and non-resident aliens: require Compliance department approval.
All approvals must be documented in the account opening workflow system.
"""

print(f"Document length: {len(SAMPLE_POLICY_DOCUMENT.split())} words")
print(f"Approximate tokens: {len(SAMPLE_POLICY_DOCUMENT) // 4}")

In [ ]:
# ── 2B: Fixed-size chunking ────────────────────────────────────────────────
def fixed_size_chunk(text: str, chunk_chars: int = 400, overlap_chars: int = 50) -> List[Dict]:
    """Split text every N characters with overlap."""
    chunks = []
    i = 0
    while i < len(text):
        chunk = text[i:i + chunk_chars]
        chunks.append({"text": chunk, "start": i, "end": i + len(chunk)})
        i += chunk_chars - overlap_chars
    return chunks

fixed_chunks = fixed_size_chunk(SAMPLE_POLICY_DOCUMENT)

print("=== FIXED-SIZE CHUNKING (400 chars, 50 overlap) ===")
print(f"Chunks created: {len(fixed_chunks)}\n")

# Show the problem: section 3.4 is split mid-content
for i, chunk in enumerate(fixed_chunks[2:5], 3):  # Show middle chunks
    print(f"--- Chunk {i} ---")
    print(chunk['text'][:300])
    print()

print("\n⚠️  Problem: Section 3.4 requirements are split across multiple chunks")
print("   Query 'business account requirements' may miss half the answer!")

In [ ]:
# ── 2C: Semantic chunking (split at section boundaries) ───────────────────
import re

def semantic_chunk_policy(text: str) -> List[Dict]:
    """Split at section headers — preserves complete policy sections."""
    section_pattern = re.compile(r'(?=^\d+\.\d+\s)', re.MULTILINE)
    parts = section_pattern.split(text)
    chunks = []
    for i, part in enumerate(parts):
        part = part.strip()
        if not part or len(part) < 50:
            continue
        # Extract section number
        section_match = re.match(r'^(\d+\.\d+)', part)
        section_num = section_match.group(1) if section_match else "intro"
        chunks.append({
            "text": part,
            "section": section_num,
            "char_count": len(part),
            "approx_tokens": len(part) // 4
        })
    return chunks

semantic_chunks = semantic_chunk_policy(SAMPLE_POLICY_DOCUMENT)

print("=== SEMANTIC CHUNKING (section-based) ===")
print(f"Chunks created: {len(semantic_chunks)}\n")

for chunk in semantic_chunks:
    print(f"Section {chunk['section']}: {chunk['approx_tokens']} tokens")
    print(f"  Preview: {chunk['text'][:120].strip()}...")
    print()

print("✅ Each section is a complete, self-contained chunk")
print("   Query 'business account requirements' → retrieves FULL Section 3.4")

In [ ]:
# ── 2D: Head-to-head comparison ───────────────────────────────────────────
QUERY = "What documents are needed for business account opening?"
query_vec = embed(QUERY)

# Search in fixed-size chunks
fixed_vecs = embed_batch([c['text'] for c in fixed_chunks])
fixed_scores = [(cosine_sim(query_vec, v), c['text'][:120]) for v, c in zip(fixed_vecs, fixed_chunks)]
fixed_top = sorted(fixed_scores, key=lambda x: x[0], reverse=True)[:3]

# Search in semantic chunks
semantic_vecs = embed_batch([c['text'] for c in semantic_chunks])
semantic_scores = [(cosine_sim(query_vec, v), c['section'], c['text'][:120])
                   for v, c in zip(semantic_vecs, semantic_chunks)]
semantic_top = sorted(semantic_scores, key=lambda x: x[0], reverse=True)[:3]

print(f"Query: '{QUERY}'\n")
print("=== FIXED-SIZE: Top 3 Results ===")
for score, preview in fixed_top:
    print(f"  [{score:.3f}] {preview}...")
print()
print("=== SEMANTIC: Top 3 Results ===")
for score, section, preview in semantic_top:
    print(f"  [{score:.3f}] Section {section}: {preview}...")

print("\n📌 Semantic chunks return complete policy sections — no information loss")

---
## Demo 3: Vector Search — Build a Banking Policy Index

**Concept:** Store embeddings in ChromaDB, search by meaning  
**Banking context:** KYC chatbot — find relevant policy for any employee question

In [ ]:
# ── 3A: Index all policy chunks into ChromaDB ─────────────────────────────
import chromadb

chroma = chromadb.EphemeralClient()  # In-memory, no files needed

# Delete if exists (for clean re-runs)
try:
    chroma.delete_collection("banking_policies")
except:
    pass

collection = chroma.create_collection(
    name="banking_policies",
    metadata={"hnsw:space": "cosine"}
)

# Embed all chunks
print("Embedding 11 policy chunks...")
t0 = time.time()
chunk_embeddings = embed_batch(BANKING_POLICY_CHUNKS)
elapsed = time.time() - t0
print(f"✅ Embedded {len(BANKING_POLICY_CHUNKS)} chunks in {elapsed:.2f}s (local, free!)")

# Store in ChromaDB
collection.add(
    ids=[f"chunk_{i}" for i in range(len(BANKING_POLICY_CHUNKS))],
    documents=BANKING_POLICY_CHUNKS,
    embeddings=chunk_embeddings.tolist(),
    metadatas=CHUNK_METADATA
)
print(f"✅ Indexed into ChromaDB — {collection.count()} vectors stored")

In [ ]:
# ── 3B: Run policy queries and inspect results ─────────────────────────────
def vector_search(query: str, n: int = 3, department_filter: str = None) -> List[Dict]:
    """Semantic search over banking policy chunks."""
    q_vec = embed(query).tolist()
    where = {"department": department_filter} if department_filter else None

    results = collection.query(
        query_embeddings=[q_vec],
        n_results=n,
        where=where,
        include=["documents", "distances", "metadatas"]
    )

    return [
        {
            "content":    doc,
            "similarity": round(1 - dist, 3),
            "section":    meta.get("section", "?"),
            "topic":      meta.get("topic", "?"),
            "doc_id":     meta.get("doc_id", "?"),
        }
        for doc, dist, meta in zip(
            results["documents"][0],
            results["distances"][0],
            results["metadatas"][0],
        )
    ]

# --- Run a battery of banking queries ---
BANKING_QUERIES = [
    "What documents do I need to open a business account?",
    "When must we file a CTR?",
    "How do we handle PEP customers?",
    "What are the requirements for suspicious activity reports?",
    "How long do we keep KYC records?",
    "OFAC screening process",
    "Can we waive an overdraft fee?",
]

console.print("[bold cyan]Banking Policy Vector Search Results[/bold cyan]\n")

for query in BANKING_QUERIES:
    results = vector_search(query, n=2)
    print(f"Q: {query}")
    for r in results:
        print(f"   [{r['similarity']:.3f}] Section {r['section']}: {r['content'][:100]}...")
    print()

In [ ]:
# ── 3C: Metadata filtering (department isolation = security) ──────────────
print("=== Metadata Filtering Demo: Department Isolation ===")
print("Scenario: Retail branch employee should NOT see Compliance-only policies\n")

query = "customer account requirements"

print(f"Query: '{query}'")
print()

print("WITHOUT filter (all departments):")
for r in vector_search(query, n=3):
    print(f"  [{r['similarity']:.3f}] Sec {r['section']} ({r['topic']}): {r['content'][:80]}...")

print()
print("WITH filter (retail department only):")
for r in vector_search(query, n=3, department_filter="retail"):
    print(f"  [{r['similarity']:.3f}] Sec {r['section']} ({r['topic']}): {r['content'][:80]}...")

print("\n📌 Metadata filtering = row-level security in banking RAG")
print("   Retail staff see only retail policies — compliance docs hidden")

---
## Demo 4: Hybrid Search — Vector + BM25 Keyword

**Concept:** Exact regulatory terms need keyword matching; semantics alone isn't enough  
**Banking context:** "Regulation E" won't be found by pure vector search

In [ ]:
from rank_bm25 import BM25Okapi

# Pre-build BM25 index over all chunks
tokenized_chunks = [chunk.lower().split() for chunk in BANKING_POLICY_CHUNKS]
bm25_index = BM25Okapi(tokenized_chunks)

def bm25_search(query: str, n: int = 5) -> List[Dict]:
    """BM25 keyword search."""
    scores = bm25_index.get_scores(query.lower().split())
    top_idx = np.argsort(scores)[-n:][::-1]
    max_score = scores.max() or 1.0
    return [
        {
            "content":    BANKING_POLICY_CHUNKS[i],
            "bm25_score": round(float(scores[i] / max_score), 3),  # Normalized 0-1
            "section":    CHUNK_METADATA[i]["section"],
            "topic":      CHUNK_METADATA[i]["topic"],
        }
        for i in top_idx if scores[i] > 0
    ]

def hybrid_search(query: str, alpha: float = 0.7, n: int = 5) -> List[Dict]:
    """
    Combine vector + BM25. alpha=1.0 → pure vector, alpha=0.0 → pure BM25.
    Reciprocal Rank Fusion approach.
    """
    # Vector scores
    q_vec = embed(query)
    vec_scores = np.array([cosine_sim(q_vec, embed(c)) for c in BANKING_POLICY_CHUNKS])

    # BM25 scores (normalized)
    bm25_raw = bm25_index.get_scores(query.lower().split())
    bm25_max = bm25_raw.max() or 1.0
    bm25_scores = bm25_raw / bm25_max

    # Combine
    combined = alpha * vec_scores + (1 - alpha) * bm25_scores
    top_idx = np.argsort(combined)[-n:][::-1]

    return [
        {
            "content":      BANKING_POLICY_CHUNKS[i],
            "hybrid_score": round(float(combined[i]), 3),
            "vec_score":    round(float(vec_scores[i]), 3),
            "bm25_score":   round(float(bm25_scores[i]), 3),
            "section":      CHUNK_METADATA[i]["section"],
            "topic":        CHUNK_METADATA[i]["topic"],
        }
        for i in top_idx
    ]

print("✅ BM25 index and hybrid search ready")

In [ ]:
# ── 4A: Compare pure vector vs hybrid for exact regulatory terms ──────────
test_queries = [
    # (query, alpha, reason)
    ("CTR filing $10,000",           0.4, "Exact regulatory threshold — BM25 helps"),
    ("SAR 30 days deadline",         0.4, "Exact deadline — BM25 critical"),
    ("enhanced due diligence",       0.7, "Semantic concept — vector sufficient"),
    ("beneficial ownership 25%",     0.5, "Mix of exact number + concept"),
]

for query, alpha, reason in test_queries:
    vec_top = vector_search(query, n=1)
    hyb_top = hybrid_search(query, alpha=alpha, n=1)

    print(f"Query: '{query}'")
    print(f"Reason: {reason}")
    print(f"  Vector  [{vec_top[0]['similarity']:.3f}]: Sec {vec_top[0]['section']} — {vec_top[0]['content'][:80]}...")
    print(f"  Hybrid  [{hyb_top[0]['hybrid_score']:.3f}]: Sec {hyb_top[0]['section']} — {hyb_top[0]['content'][:80]}...")
    same = vec_top[0]['section'] == hyb_top[0]['section']
    print(f"  Same result: {'Yes ✓' if same else 'No — hybrid found different (potentially better) result'}")
    print()

In [ ]:
# ── 4B: Alpha tuning visualization ────────────────────────────────────────
# Show how different alpha values change which document ranks #1
query = "CTR FinCEN Form 104 $10,000 cash reporting"

print(f"Query: '{query}'")
print(f"\nHow alpha affects results (0=pure BM25, 1=pure vector):")
print(f"{'Alpha':<8} {'Top Section':<12} {'Hybrid Score':<14} {'Vec Score':<12} {'BM25 Score'}")
print("-" * 60)

for alpha in [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]:
    result = hybrid_search(query, alpha=alpha, n=1)[0]
    print(f"{alpha:<8.1f} {result['section']:<12} {result['hybrid_score']:<14.3f} "
          f"{result['vec_score']:<12.3f} {result['bm25_score']:.3f}")

print("\n📌 For regulatory exact-term queries: alpha=0.3-0.5 (more BM25 weight)")
print("   For general policy questions: alpha=0.7-0.9 (more vector weight)")

---
## Demo 5: Complete RAG Pipeline with Claude

**Concept:** Retrieved chunks + Claude = cited, accurate banking answers  
**Requires:** `ANTHROPIC_API_KEY` environment variable

In [ ]:
import anthropic

# Set your key: os.environ["ANTHROPIC_API_KEY"] = "sk-ant-..."
claude = anthropic.Anthropic()  # Reads ANTHROPIC_API_KEY from env

def banking_rag(query: str,
                top_k: int = 3,
                use_hybrid: bool = True,
                department_filter: str = None) -> Dict:
    """
    Complete RAG pipeline:
      1. Retrieve relevant chunks (hybrid or vector)
      2. Build prompt with citations
      3. Generate answer with Claude
    """
    t0 = time.time()

    # Step 1: Retrieve
    if use_hybrid:
        chunks = hybrid_search(query, alpha=0.6, n=top_k)
        method = "hybrid"
    else:
        chunks = vector_search(query, n=top_k, department_filter=department_filter)
        method = "vector"
    retrieve_ms = (time.time() - t0) * 1000

    # Step 2: Build context with citations
    context_blocks = [
        f"[Source {i+1}: {c.get('doc_id','POL-KYC-2024')}, "
        f"Section {c['section']}]\n{c['content']}"
        for i, c in enumerate(chunks)
    ]
    context = "\n\n---\n\n".join(context_blocks)

    # Step 3: Generate with Claude
    t1 = time.time()
    response = claude.messages.create(
        model="claude-haiku-4-5-20251001",  # Fast + cheap for demos
        max_tokens=400,
        system=(
            "You are a banking compliance assistant. "
            "Answer using ONLY the provided policy sources. "
            "Always cite the Source number and Section. "
            "If the answer is not in the sources, say so clearly."
        ),
        messages=[{
            "role": "user",
            "content": f"Policy Sources:\n{context}\n\nQuestion: {query}\n\nAnswer:"
        }]
    )
    generate_ms = (time.time() - t1) * 1000
    total_ms = (time.time() - t0) * 1000

    answer = response.content[0].text
    input_tokens  = response.usage.input_tokens
    output_tokens = response.usage.output_tokens
    cost = (input_tokens * 0.00025 + output_tokens * 0.00125) / 1000  # Haiku pricing

    return {
        "query":        query,
        "answer":       answer,
        "chunks_used":  len(chunks),
        "method":       method,
        "retrieve_ms":  round(retrieve_ms),
        "generate_ms":  round(generate_ms),
        "total_ms":     round(total_ms),
        "input_tokens": input_tokens,
        "output_tokens":output_tokens,
        "cost_usd":     round(cost, 5),
        "sources":      [(c['section'], c['topic']) for c in chunks],
    }

print("✅ RAG pipeline ready")

In [ ]:
# ── 5A: Run banking policy Q&A ────────────────────────────────────────────
questions = [
    "What documents are required to open a business account?",
    "When must we file a Currency Transaction Report?",
    "How should we handle a customer who is a PEP?",
]

for question in questions:
    print("=" * 70)
    result = banking_rag(question)
    print(f"Q: {result['query']}")
    print(f"\nA: {result['answer']}")
    print(f"\n📊 Sources used: {result['sources']}")
    print(f"⚡ Latency: {result['total_ms']}ms "
          f"(retrieve: {result['retrieve_ms']}ms, generate: {result['generate_ms']}ms)")
    print(f"💰 Cost: ${result['cost_usd']:.5f} | "
          f"Tokens: {result['input_tokens']} in / {result['output_tokens']} out")
    print()

In [ ]:
# ── 5B: What happens when the answer is NOT in the database? ──────────────
out_of_scope = [
    "What is the current Fed Funds rate?",
    "How do I process a mortgage application?",
]

print("=== RAG Handles Unknown Questions Gracefully ===")
for q in out_of_scope:
    result = banking_rag(q)
    print(f"Q: {q}")
    print(f"A: {result['answer']}")
    print(f"Sources searched: {result['sources']}")
    print()

print("📌 Claude correctly says it cannot answer from the provided sources")
print("   This is the right behavior — no hallucination!")

---
## Demo 6: RAG Failure Modes — Learn to Debug

**Concept:** See what goes wrong and WHY — then fix it  
**Banking context:** Common failures in compliance chatbots

In [ ]:
# ── 6A: Failure Mode 1 — Wrong chunk size (too large) ────────────────────
print("=" * 60)
print("FAILURE 1: Chunk size too large (noisy context)")
print("=" * 60)

# Simulate large chunks by concatenating 3 real chunks together
LARGE_CHUNKS = [
    " ".join(BANKING_POLICY_CHUNKS[i:i+3])
    for i in range(0, len(BANKING_POLICY_CHUNKS), 3)
]

large_vecs = embed_batch(LARGE_CHUNKS)
query = "CTR filing deadline"
q_vec = embed(query)

# Search large chunks
large_scores = [(cosine_sim(q_vec, v), chunk[:200]) for v, chunk in zip(large_vecs, LARGE_CHUNKS)]
large_top = sorted(large_scores, reverse=True)[0]

# Search normal chunks
normal_scores = [(cosine_sim(q_vec, embed(c)), c[:200]) for c in BANKING_POLICY_CHUNKS]
normal_top = sorted(normal_scores, reverse=True)[0]

print(f"Query: '{query}'\n")
print(f"LARGE chunks (1500+ chars): score={large_top[0]:.3f}")
print(f"  Content: {large_top[1]}...")
print()
print(f"NORMAL chunks (~500 chars): score={normal_top[0]:.3f}")
print(f"  Content: {normal_top[1]}...")
print()
print("📌 Large chunks dilute the relevant signal with unrelated content")
print("   Lower similarity score → may not rank #1 → missed retrieval")

In [ ]:
# ── 6B: Failure Mode 2 — No overlap (concept split at boundary) ───────────
print("=" * 60)
print("FAILURE 2: Zero overlap — concepts split at boundaries")
print("=" * 60)

# Create a document where key info straddles two fixed chunks
doc_with_split = (
    "The Bank Secrecy Act requires financial institutions to maintain detailed records "
    "of all customer transactions. Compliance officers must ensure that all reporting "
    "thresholds are properly monitored. " * 5 +  # Padding to push next sentence to chunk 2
    "Currency Transaction Reports must be filed for cash transactions exceeding $10,000 "
    "within 15 calendar days of the transaction date."
)

chunk_size = 400

# No overlap
no_overlap = [
    doc_with_split[i:i+chunk_size]
    for i in range(0, len(doc_with_split), chunk_size)
]

# With overlap
overlap = 80
with_overlap = [
    doc_with_split[i:i+chunk_size]
    for i in range(0, len(doc_with_split), chunk_size - overlap)
]

# Find the chunk containing CTR info in each
ctr_query = embed("CTR filing $10,000 deadline")

no_overlap_scores = [cosine_sim(ctr_query, embed(c)) for c in no_overlap]
with_overlap_scores = [cosine_sim(ctr_query, embed(c)) for c in with_overlap]

print(f"Query: 'CTR filing $10,000 deadline'")
print(f"\nNO OVERLAP chunks ({len(no_overlap)} chunks):")
for i, (score, chunk) in enumerate(zip(no_overlap_scores, no_overlap)):
    ctr_present = '$10,000' in chunk or 'CTR' in chunk
    print(f"  Chunk {i+1}: score={score:.3f} | CTR info present: {ctr_present}")

print(f"\nWITH OVERLAP ({overlap} chars) chunks ({len(with_overlap)} chunks):")
for i, (score, chunk) in enumerate(zip(with_overlap_scores, with_overlap)):
    ctr_present = '$10,000' in chunk or 'CTR' in chunk
    print(f"  Chunk {i+1}: score={score:.3f} | CTR info present: {ctr_present}")

print("\n📌 Overlap ensures key sentences appear in multiple chunks")
print("   Recommendation: 10-20% overlap for banking policy documents")

In [ ]:
# ── 6C: Failure Mode 3 — K too high (context flooding) ───────────────────
print("=" * 60)
print("FAILURE 3: K too high — noisy context causes confusion")
print("=" * 60)

query = "How do I open a business account?"

print(f"Query: '{query}'\n")
for k in [1, 3, 5, 8]:
    results = vector_search(query, n=k)
    relevant = sum(1 for r in results if r['topic'] in ['account_opening'])
    precision = relevant / k
    total_chars = sum(len(r['content']) for r in results)
    print(f"K={k}: Precision={precision:.0%} | "
          f"Relevant={relevant}/{k} | "
          f"Context size={total_chars} chars | "
          f"Topics: {[r['topic'] for r in results]}")

print("\n📌 K=3-5 is the sweet spot for banking compliance queries")
print("   K>5 introduces noise: overdraft policy mixed into account-opening answer")

In [ ]:
# ── 6D: Debug helper — show exactly what RAG sees ─────────────────────────
def debug_retrieval(query: str, n: int = 5):
    """Full retrieval debug: scores, content, topics."""
    q_vec = embed(query)
    all_scores = [
        (cosine_sim(q_vec, embed(c)), i, c[:120], CHUNK_METADATA[i])
        for i, c in enumerate(BANKING_POLICY_CHUNKS)
    ]
    all_scores.sort(reverse=True)

    console.print(f"\n[bold cyan]Debug Retrieval: '{query}'[/bold cyan]")
    t = Table(show_header=True)
    t.add_column("Rank", justify="right", width=5)
    t.add_column("Score", justify="right", width=7)
    t.add_column("Section", width=8)
    t.add_column("Topic", width=16)
    t.add_column("Preview", max_width=60)

    for rank, (score, idx, preview, meta) in enumerate(all_scores, 1):
        color = "green" if score > 0.6 else "yellow" if score > 0.45 else "red"
        t.add_row(
            str(rank),
            f"[{color}]{score:.3f}[/{color}]",
            meta["section"],
            meta["topic"],
            preview + "..."
        )
        if rank == n:
            t.add_row("...", "---", "---", "---", f"({len(all_scores)-n} more not shown)")
            break

    console.print(t)

# Run debug on a few queries
debug_retrieval("CTR filing requirements")
debug_retrieval("employee leave policy")  # Not in our KB — watch scores drop

---
## Demo 7: Advanced Retrieval — Re-ranking, HyDE, Contextual RAG

**Concept:** Push precision above 85% with post-retrieval techniques  
**Banking context:** Critical compliance queries where accuracy is non-negotiable

In [ ]:
# ── 7A: Re-ranking with cross-encoder ─────────────────────────────────────
# Cross-encoders score (query, document) PAIRS — slower but more accurate

try:
    from sentence_transformers import CrossEncoder
    reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
    RERANKER_AVAILABLE = True
    print("✅ Cross-encoder loaded (downloads ~70MB once)")
except Exception as e:
    print(f"⚠️  Cross-encoder not available: {e}")
    print("   Install: pip install sentence-transformers")
    RERANKER_AVAILABLE = False

def retrieve_and_rerank(query: str, initial_k: int = 8, final_k: int = 3) -> List[Dict]:
    """
    Two-stage retrieval:
      Stage 1: Fast vector search (broad)
      Stage 2: Cross-encoder re-ranking (precise)
    """
    # Stage 1: Broad retrieval
    initial = vector_search(query, n=initial_k)

    if not RERANKER_AVAILABLE:
        return initial[:final_k]

    # Stage 2: Re-rank with cross-encoder
    pairs = [(query, r['content']) for r in initial]
    rerank_scores = reranker.predict(pairs)

    # Sort by cross-encoder score
    reranked = sorted(
        zip(rerank_scores, initial),
        key=lambda x: x[0],
        reverse=True
    )

    return [
        {**item, "rerank_score": round(float(score), 3)}
        for score, item in reranked[:final_k]
    ]

# Compare: vector vs reranked
query = "suspicious activity SAR filing deadline compliance"
print(f"Query: '{query}'\n")

print("Vector search (top 3):")
for r in vector_search(query, n=3):
    print(f"  [{r['similarity']:.3f}] Sec {r['section']} ({r['topic']}): {r['content'][:80]}...")

print()
print("After re-ranking (top 3 from initial 8):")
for r in retrieve_and_rerank(query, initial_k=8, final_k=3):
    score_str = f"{r.get('rerank_score', r['similarity']):.3f}"
    print(f"  [{score_str}] Sec {r['section']} ({r['topic']}): {r['content'][:80]}...")

In [ ]:
# ── 7B: HyDE — Hypothetical Document Embeddings ───────────────────────────
# Insight: A hypothetical answer is MORE similar to real docs than the question itself

def hyde_retrieve(question: str, n: int = 3) -> List[Dict]:
    """Generate a hypothetical answer, embed it, search with that embedding."""

    # Step 1: Generate hypothetical answer with Claude
    response = claude.messages.create(
        model="claude-haiku-4-5-20251001",
        max_tokens=150,
        messages=[{
            "role": "user",
            "content": (
                f"You are a banking compliance expert. Write a concise, "
                f"factual 2-3 sentence answer to this question as if from "
                f"official banking policy. Include specific numbers and terms.\n\n"
                f"Question: {question}\n\nAnswer:"
            )
        }]
    )
    hypothetical = response.content[0].text

    # Step 2: Embed the hypothetical answer
    hyp_vec = embed(hypothetical)

    # Step 3: Search using hypothetical answer embedding
    q_vec_scores = [
        (cosine_sim(hyp_vec, embed(c)), i)
        for i, c in enumerate(BANKING_POLICY_CHUNKS)
    ]
    top = sorted(q_vec_scores, reverse=True)[:n]

    return {
        "hypothetical_answer": hypothetical,
        "results": [
            {
                "content":    BANKING_POLICY_CHUNKS[i],
                "similarity": round(score, 3),
                "section":    CHUNK_METADATA[i]["section"],
                "topic":      CHUNK_METADATA[i]["topic"],
            }
            for score, i in top
        ]
    }

# Demo
question = "What is the exact dollar threshold for filing a Currency Transaction Report?"

print(f"Question: '{question}'\n")

# Standard search
std = vector_search(question, n=1)
print(f"Standard vector search top result: [{std[0]['similarity']:.3f}]")
print(f"  {std[0]['content'][:120]}...")

# HyDE search
hyde = hyde_retrieve(question, n=1)
print(f"\nHyDE hypothetical answer generated:")
print(f"  '{hyde['hypothetical_answer']}'")
print(f"\nHyDE top result: [{hyde['results'][0]['similarity']:.3f}]")
print(f"  {hyde['results'][0]['content'][:120]}...")

std_score = std[0]['similarity']
hyde_score = hyde['results'][0]['similarity']
print(f"\n📌 HyDE similarity: {hyde_score:.3f} vs Standard: {std_score:.3f} "
      f"({'better' if hyde_score > std_score else 'similar'})")

In [ ]:
# ── 7C: Contextual RAG — add context to chunks before embedding ───────────
# Anthropic research: prepending document context reduces retrieval failures by 35%

def add_chunk_context(chunk: str, doc_title: str, section_title: str) -> str:
    """
    Prepend context so isolated chunks carry document-level information.
    This makes each chunk self-describing.
    """
    context = (
        f"This excerpt is from {doc_title}. "
        f"It covers {section_title}. "
    )
    return context + chunk

SECTION_TITLES = {
    "3.4": "business account opening requirements and FinCEN CDD compliance",
    "3.5": "individual retail account opening identity verification",
    "4.2": "Currency Transaction Report CTR filing thresholds and deadlines",
    "5.1": "Politically Exposed Person PEP enhanced due diligence",
    "6.3": "Suspicious Activity Report SAR filing obligations",
    "4.5": "international wire transfer monitoring and Travel Rule compliance",
    "8.2": "KYC record retention requirements",
    "9.1": "customer risk rating framework CDD",
    "11.4": "overdraft fee waiver policy",
    "2.1": "Basel III capital adequacy CET1 requirements",
    "7.1": "OFAC sanctions compliance SDN screening",
}

# Create contextual versions of all chunks
CONTEXTUAL_CHUNKS = [
    add_chunk_context(
        chunk,
        doc_title=meta["doc_id"],
        section_title=SECTION_TITLES.get(meta["section"], meta["topic"])
    )
    for chunk, meta in zip(BANKING_POLICY_CHUNKS, CHUNK_METADATA)
]

# Embed contextual chunks
contextual_vecs = embed_batch(CONTEXTUAL_CHUNKS)
original_vecs   = embed_batch(BANKING_POLICY_CHUNKS)

# Compare on ambiguous short query
short_queries = [
    "reporting threshold",       # Could be CTR or SAR
    "ownership documentation",   # Could be business or trust
    "filing deadline",           # Could be CTR, SAR, or OFAC
]

print("=== Contextual RAG: Before vs After Adding Context ===")
for q in short_queries:
    q_vec = embed(q)

    orig_scores  = [(cosine_sim(q_vec, v), CHUNK_METADATA[i]["section"])
                    for i, v in enumerate(original_vecs)]
    ctx_scores   = [(cosine_sim(q_vec, v), CHUNK_METADATA[i]["section"])
                    for i, v in enumerate(contextual_vecs)]

    orig_top  = sorted(orig_scores,  reverse=True)[0]
    ctx_top   = sorted(ctx_scores,   reverse=True)[0]

    print(f"\nQuery: '{q}'")
    print(f"  Original   → Sec {orig_top[1]} (score: {orig_top[0]:.3f})")
    print(f"  Contextual → Sec {ctx_top[1]}  (score: {ctx_top[0]:.3f})",
          "✓ improved" if ctx_top[0] > orig_top[0] else "")

print("\n📌 Contextual RAG: prepend 2-3 sentences of document context to each chunk")
print("   Anthropic reports 35% fewer retrieval failures with this technique")

---
## Demo 8: Evaluation — Measure RAG Quality

**Concept:** Quantify how good your RAG system is with real metrics  
**Banking context:** Build a golden test set and track quality over time

In [ ]:
# ── 8A: Golden test set — query → expected sections ───────────────────────
# In production: compliance experts label these

GOLDEN_TEST_SET = [
    {
        "query":            "What documents are required for business account opening?",
        "relevant_sections": {"3.4"},  # Section 3.4 is the correct answer
    },
    {
        "query":            "When must a CTR be filed?",
        "relevant_sections": {"4.2"},
    },
    {
        "query":            "How do we handle PEP customers?",
        "relevant_sections": {"5.1"},
    },
    {
        "query":            "What are the SAR filing requirements?",
        "relevant_sections": {"6.3"},
    },
    {
        "query":            "OFAC sanctions screening process",
        "relevant_sections": {"7.1"},
    },
    {
        "query":            "Record retention for wire transfers and CTR",
        "relevant_sections": {"8.2", "4.2"},  # Both sections relevant
    },
    {
        "query":            "Can we waive an overdraft fee for a good customer?",
        "relevant_sections": {"11.4"},
    },
    {
        "query":            "Basel III CET1 capital ratio requirement",
        "relevant_sections": {"2.1"},
    },
]

def evaluate_retrieval(test_set: List[Dict], search_fn, k: int = 3) -> Dict:
    """Calculate Precision@K and Recall@K over a golden test set."""
    precisions, recalls, mrrs = [], [], []

    for test in test_set:
        query    = test["query"]
        relevant = test["relevant_sections"]
        results  = search_fn(query, n=k)

        retrieved_sections = [r["section"] for r in results]

        hits = sum(1 for s in retrieved_sections if s in relevant)
        precision = hits / k
        recall    = hits / len(relevant)
        precisions.append(precision)
        recalls.append(recall)

        # MRR: reciprocal rank of first relevant result
        mrr = 0.0
        for rank, section in enumerate(retrieved_sections, 1):
            if section in relevant:
                mrr = 1.0 / rank
                break
        mrrs.append(mrr)

    return {
        f"Precision@{k}": round(np.mean(precisions), 3),
        f"Recall@{k}":    round(np.mean(recalls), 3),
        "MRR":           round(np.mean(mrrs), 3),
    }

# Evaluate different strategies
K = 3

vector_metrics = evaluate_retrieval(
    GOLDEN_TEST_SET,
    lambda q, n: vector_search(q, n=n),
    k=K
)
hybrid_metrics = evaluate_retrieval(
    GOLDEN_TEST_SET,
    lambda q, n: hybrid_search(q, alpha=0.6, n=n),
    k=K
)
rerank_metrics = evaluate_retrieval(
    GOLDEN_TEST_SET,
    lambda q, n: retrieve_and_rerank(q, initial_k=8, final_k=n),
    k=K
)

# Display comparison
console.print("\n[bold cyan]RAG Strategy Comparison (Banking Golden Test Set)[/bold cyan]\n")

table = Table(show_header=True)
table.add_column("Strategy",         style="cyan",   width=22)
table.add_column(f"Precision@{K}",   style="green",  justify="right")
table.add_column(f"Recall@{K}",      style="yellow", justify="right")
table.add_column("MRR",              style="magenta",justify="right")
table.add_column("Approx Latency",   style="white",  justify="right")

strategies = [
    ("Vector Search",          vector_metrics, "~5ms"),
    ("Hybrid (α=0.6)",         hybrid_metrics, "~10ms"),
    ("Hybrid + Re-rank",        rerank_metrics, "~200ms"),
]

for name, metrics, latency in strategies:
    table.add_row(
        name,
        str(metrics[f"Precision@{K}"]),
        str(metrics[f"Recall@{K}"]),
        str(metrics["MRR"]),
        latency
    )

console.print(table)
print("\n📌 Higher is better. Banking compliance target: Precision@3 > 0.80, MRR > 0.80")

In [ ]:
# ── 8B: Per-query breakdown — find where retrieval fails ──────────────────
print("=== Per-Query Retrieval Analysis ===")
print(f"{'Query':<50} {'Expected':<10} {'Got':<25} {'Pass'}")
print("-" * 100)

for test in GOLDEN_TEST_SET:
    results = hybrid_search(test["query"], alpha=0.6, n=3)
    retrieved_sections = set(r["section"] for r in results)
    expected = test["relevant_sections"]
    hit = bool(expected & retrieved_sections)  # At least one relevant section found

    status = "✅" if hit else "❌"
    print(f"{test['query'][:50]:<50} "
          f"{str(expected):<10} "
          f"{str(retrieved_sections):<25} "
          f"{status}")

print("\n📌 Failed queries reveal where to improve:")
print("   → Wrong embedding model (domain mismatch)")
print("   → Chunk too large (signal diluted)")
print("   → Missing metadata (wrong department filter)")

In [ ]:
# ── 8C: Cost and latency benchmark ────────────────────────────────────────
print("=== Cost & Latency Benchmark ===")
print("Simulating 50 queries across strategies\n")

N_QUERIES = 50
TEST_QUERIES = [
    "CTR filing requirements",
    "PEP customer handling",
    "SAR deadline",
    "business account documents",
    "OFAC screening",
] * 10  # 50 queries

def benchmark_strategy(name: str, search_fn, queries: List[str]):
    times = []
    for q in queries:
        t0 = time.time()
        search_fn(q)
        times.append((time.time() - t0) * 1000)
    return {
        "strategy": name,
        "avg_ms":   round(np.mean(times), 1),
        "p95_ms":   round(np.percentile(times, 95), 1),
        "embed_cost_1k_queries": "$0.00 (local)",
    }

results = [
    benchmark_strategy("Vector (K=3)",       lambda q: vector_search(q, n=3),           TEST_QUERIES),
    benchmark_strategy("Hybrid (K=3)",       lambda q: hybrid_search(q, n=3),           TEST_QUERIES),
    benchmark_strategy("Hybrid + Rerank",    lambda q: retrieve_and_rerank(q, 8, 3),    TEST_QUERIES),
]

table = Table(title="Retrieval Strategy Benchmark (50 queries)")
table.add_column("Strategy",          style="cyan")
table.add_column("Avg Latency",       style="green",  justify="right")
table.add_column("P95 Latency",       style="yellow", justify="right")
table.add_column("Embed Cost/1K",     style="magenta")

for r in results:
    table.add_row(r["strategy"], f"{r['avg_ms']}ms", f"{r['p95_ms']}ms",
                  r["embed_cost_1k_queries"])

console.print(table)
print("\n📌 sentence-transformers runs locally — zero embedding API cost")
print("   In production with OpenAI embeddings: ~$0.02 per 1M tokens")

---
## Hands-On Exercise: Build Your Own Banking RAG

**Scenario:** The compliance team needs a SAR/CTR decision helper for call center agents.

**Your task:** Extend the RAG system to:
1. Add 3 new policy chunks (write them yourself)
2. Choose the best retrieval strategy
3. Tune K for best precision
4. Test with 5 realistic call center queries
5. Measure Precision@K on your test set

In [ ]:
# ── Exercise: Your Turn ────────────────────────────────────────────────────

# STEP 1: Add your own policy chunks (write realistic banking policy text)
MY_POLICY_CHUNKS = [
    # TODO: Write 3 realistic banking policy chunks below
    "Section X.X: [Your policy text here...]",
    "Section X.X: [Your policy text here...]",
    "Section X.X: [Your policy text here...]",
]

MY_METADATA = [
    {"section": "X.1", "topic": "your_topic", "department": "compliance", "doc_id": "POL-MY-2024"},
    {"section": "X.2", "topic": "your_topic", "department": "compliance", "doc_id": "POL-MY-2024"},
    {"section": "X.3", "topic": "your_topic", "department": "compliance", "doc_id": "POL-MY-2024"},
]

# STEP 2: Add to existing collection
all_chunks = BANKING_POLICY_CHUNKS + MY_POLICY_CHUNKS
all_metadata = CHUNK_METADATA + MY_METADATA
all_vecs = embed_batch(all_chunks)
print(f"Total chunks in knowledge base: {len(all_chunks)}")

# STEP 3: Test your queries
MY_TEST_QUERIES = [
    # TODO: Write 5 realistic call center questions
    "Question 1",
    "Question 2",
    "Question 3",
    "Question 4",
    "Question 5",
]

print("\n=== Your RAG Results ===")
for q in MY_TEST_QUERIES:
    q_vec = embed(q)
    scores = [(cosine_sim(q_vec, v), i) for i, v in enumerate(all_vecs)]
    top = sorted(scores, reverse=True)[:3]

    print(f"\nQ: {q}")
    for score, idx in top:
        meta = all_metadata[idx]
        print(f"  [{score:.3f}] Sec {meta['section']}: {all_chunks[idx][:100]}...")

print("\n✅ Exercise complete! Discuss with your team:")
print("   - Which chunk size worked best?")
print("   - When did vector search fail? When did hybrid help?")
print("   - What metadata would you add for your bank's use case?")

---
## Summary: What We Built

| Demo | Concept | Key Insight |
|------|---------|-------------|
| 1. Embeddings | Text → vectors | `KYC` ≈ `Know Your Customer` in vector space |
| 2. Chunking | Split strategies | Semantic chunks preserve complete policy sections |
| 3. Vector Search | ChromaDB index | Metadata filters = department-level security |
| 4. Hybrid Search | BM25 + vector | Regulatory exact terms need keyword matching |
| 5. Full RAG | Claude generation | Cited answers from policy, no hallucination |
| 6. Failure Modes | Debug retrieval | Large chunks & no overlap cause retrieval failures |
| 7. Advanced RAG | Re-rank, HyDE, Contextual | +15-35% precision on compliance queries |
| 8. Evaluation | Precision, Recall, MRR | Build golden test set → track quality over time |

### Production Banking RAG Stack Recommendation

```
Embedding:    sentence-transformers (local) or text-embedding-3-small (OpenAI)
Chunking:     Semantic (512 tokens, 10% overlap) for policy docs
Vector DB:    pgvector (already approved in most banks) or Chroma (prototypes)
Retrieval:    Hybrid (α=0.5) + re-ranking for compliance queries
Security:     Metadata filters by department + security_classification
Generation:   Claude with citation requirements and grounding instruction
Evaluation:   Weekly Precision@3 and MRR on golden test set
```

### Next: Session 4 — Context Engineering
How to manage the context window that RAG fills — token budgeting, prompt structure, caching.